In [ ]:
import os
import sys
import time
import json
import math
import talib
import selenium
import datetime
import requests
import threading
import numpy as np
import pandas as pd
import telegram_bot
from alice_blue import *
from datetime import timedelta
from fyers_api import fyersModel
from fyers_api import accessToken
from jugaad_trader import Zerodha
from sleep_function import sleeper
from collections import namedtuple
from collections import defaultdict

#### CLASS DATA

In [ ]:
class AmadeusData():
    
    # Access user credentials from pre-stored CSV and save them into these variables
    username, password, twoFA, api_secret = pd.read_csv('alice_blue_credentials.csv').iloc[0]
    
    # -----------------------------------------------------------
    # Init Method accepts the timeframe to resample the tick data 
    # to and a dictionary 'instrument_exchange' which specifies the
    # instruent and exchange along with other required parameters
    # (refer main file to set input)
    # -----------------------------------------------------------
    
    def __init__(self, timeframe, instrument_exchange):
        
        # Create access_token using login_and_get_access_token() function with your username, password, 2FA and api_secret
        self.access_token = AliceBlue.login_and_get_access_token(username = self.username, 
                                                                 password = self.password, 
                                                                 twoFA = self.twoFA,  
                                                                 api_secret = self.api_secret)
        
        # Once you have your access_token, you can create an AliceBlue object with your access_token, username and password
        self.alice = AliceBlue(username = self.username, password = self.password, access_token = self.access_token)
    
        # Instruments are represented by instrument objects. These are named-tuples that are created while 
        # getting the master contracts. They are used when placing an order and searching for an instrument.
        self.Instrument = namedtuple('Instrument', ['exchange', 'token', 'symbol',
                                      'name', 'expiry', 'lot_size'])
    
        # Initialise socket to False until subscribed to instruments
        self.socket_opened = False
        
        # Set timeframe as a class attribute
        self.timeframe = timeframe

        # Create a default dictionary which stores empty dataframes for any default key, 
        # to store live tick data once an instrument is subscribed to
        self.raw_data = defaultdict(
            lambda: pd.DataFrame(columns = ['datetime', 'ticker', 'ltp', 'cum_volume', 'volume']).set_index('datetime'))

        # Create a default dictionary which stores empty dataframes for any default key, 
        # to store resampled data once an instrument is subscribed to
        self.data = defaultdict(
            lambda: pd.DataFrame(columns = ['datetime', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'date', 'time']).set_index('datetime'))
        
        # Set instrument_exchange dictionary input as a class attribute
        self.instrument_exchange = instrument_exchange
        
        # Check of data connection is established
        self.data_strike = 0
        self.last_checked = datetime.datetime.now()
        
    
    # -----------------------------------------------------------
    # Get Live Quote Update
    # -----------------------------------------------------------
    
    def get_ltp(self, message):
    
        # Set message output as class attribute - this would be updated once websocket is started
        self.message = message
        
        # Extract instrument name from message and set it as class attribute
        self.instrument = self.message['instrument'].symbol

        # Extract timestamp from message and set it as class attribute
        self.timestamp = datetime.datetime.fromtimestamp(self.message['exchange_time_stamp'])
        
        # Live update ticker and volume extracted from message and append it as row to the raw_data
        self.raw_data[self.instrument] = self.raw_data[self.instrument].append(
                            pd.Series({'ticker' : self.instrument, 'ltp' : self.message['ltp'], 'cum_volume' : self.message['volume']}, 
                            name = self.timestamp))
        
        # Extract volume from cumulative volume by taking difference
        self.raw_data[self.instrument]['volume'] = self.raw_data[self.instrument]['cum_volume'].diff()
        
    
    # -----------------------------------------------------------
    # Open Socket Callback
    # -----------------------------------------------------------
     
    def open_callback(self):
    
        # Set socket_opened to True
        self.socket_opened = True      
    
    # -----------------------------------------------------------
    # Start Live Data Feed and Start Websocket
    # -----------------------------------------------------------
    
    def start_live_feed(self):
    
        # Start Alice Blue Websocket and set subscribe callback to get_ltp class method and 
        # socket open callback to open_callback class method
        self.alice.start_websocket(subscribe_callback=self.get_ltp,
                          socket_open_callback=self.open_callback,
                          run_in_background=True)

        # Wait and do nothing until socket_opened turns True
        while(self.socket_opened==False):
            pass

        # Create a Zerodha object to get historical data using Jugaad Trader
        kite = Zerodha()
        # Automatically load pre-stored credentials
        kite.load_creds()
        # Login to Zerodha with loaded credentials
        kite.login()
        
        # Iterate over the keys in instrument_exchange dictionary, which are the instrument names
        for key in self.instrument_exchange:
        
            # Subscribe the particular instrument to alice blue by passing the instrument symbol name and exchange name
            self.alice.subscribe(self.alice.get_instrument_by_symbol(self.instrument_exchange[key]['exchange'], key), LiveFeedType.COMPACT)

            # Check if the particular instrument is Futures
            if (key[-3:] == 'FUT'):

                # Store the particular instrument in the form Zerodha can extract its token number
                ins_kite = self.instrument_exchange[key]['exchange'] + ':' + key.split(' ')[0] + str(datetime.date.today().year)[2:] + key.split(' ')[1] + key.split(' ')[2]
                
                # Extract the instrument's token number using quote method from Zerodha object
                ins_kite_number = kite.quote(ins_kite)[ins_kite]['instrument_token']
            
            # If the instrument is not Futures
            else:

                # Store the particular instrument in the form Zerodha can extract its token number
                ins_kite = self.instrument_exchange[key]['exchange'] + ':' + key
                
                # Extract the instrument's token number using quote method from Zerodha object
                ins_kite_number = kite.quote(ins_kite)[ins_kite]['instrument_token']
                
            # Define date from when you want historical data
            # in this case, it is from 5 days ago
            from_date = datetime.date.today() - datetime.timedelta(days = 5)

            # Define date till when you want historical data
            # in this case, it is until the current time
            to_date = datetime.date.today()

            # Define timeframe for the historical data's frequency
            interval = "minute"

            # Save historical data to class attribute data and append live resampled data to this eventually
            self.data[key] = pd.DataFrame(kite.historical_data(ins_kite_number, from_date, to_date, interval, continuous=False, oi=False))
   
            ### Clean the Zerodha historical data to match its format to AliceBlue live data ###
    
            # Create datetime column which is equal to the Zerodha historical data's date column
            self.data[key]['datetime'] = self.data[key]['date']
            
            # Create new column ticker to store symbol name
            self.data[key]['ticker'] = key

            # Reassign date column to the date extracted from datetime column
            self.data[key]['date'] = self.data[key]['datetime'].apply(lambda x : x.date())
            
            # Create time column to the time extracted from datetime column
            self.data[key]['time'] = self.data[key]['datetime'].apply(lambda x : x.time())

            # Reorder column names in the dataframe
            self.data[key] = self.data[key][['datetime','ticker','open','high','low','close','volume','date','time']]
            
            # Reassign datetime column as a combined string form of date and time from the respective columns
            self.data[key]['datetime'] = self.data[key]['date'].apply(str) + self.data[key]['time'].apply(str)
            
            # Set datetime column to datetime format using strptime method
            self.data[key]['datetime'] = self.data[key]['datetime'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d%H:%M:%S'))
            
            # Set datetime column as index
            self.data[key].set_index('datetime', inplace=True)
            
            
    # -----------------------------------------------------------
    # Static method to obtain required time delta from timeframe
    # This will be required by the data resampling function
    # Refer class method resample_data to understand requirement
    # -----------------------------------------------------------
    
    @staticmethod
    def obtain_timedelta(timeframe):

        # If the timeframe is in seconds, as indicated by the last element of the string being S
        if timeframe[-1] == 'S':
            
            # Set timedelta to seconds equal to the number specified before the last element
            td = datetime.timedelta(seconds = int(timeframe[:-1]))

        # Else If the timeframe is in minutes, as indicated by the last element of the string being T
        elif timeframe[-1] == 'T':
            
            # Set timedelta to minutes equal to the number specified before the last element
            td = datetime.timedelta(minutes = int(timeframe[:-1]))

        # Else If the timeframe is in minutes, as indicated by the last 3 elements of the string being min
        elif timeframe[-3:].lower() == 'min':
            
            # Set timedelta to minutes equal to the number specified before the last 3 elements
            td = datetime.timedelta(minutes = int(timeframe[:-3]))

        # Else If the timeframe is in hours, as indicated by the last element of the string being H
        elif timeframe[-1] == 'H':
            
            # Set timedelta to hours equal to the number specified before the last element
            td = datetime.timedelta(hours = int(timeframe[:-1]))

        # Else If the timeframe is in days, as indicated by the last element of the string being D
        elif timeframe[-1] == 'D':
            
            # Set timedelta to days equal to the number specified before the last element
            td = datetime.timedelta(days = int(timeframe[:-1]))

        # Else If the timeframe is in weeks, as indicated by the last element of the string being W
        elif timeframe[-1] == 'W':
            
            # Set timedelta to weeks equal to the number specified before the last element
            td = datetime.timedelta(weeks = int(timeframe[:-1]))

        # If none of the above conditions are met
        else:
            
            # Set timedelta to None
            td = None

        # Return the calculated timedelta
        return td
    
    
    # -----------------------------------------------------------
    # Resample Data to required timeframe
    # -----------------------------------------------------------
    
    def resample_data(self):
    
        # Set the flag to check thread run to True
        self.thread_resample_run = True
        
        # Run while the flag to check thread run is True
        while self.thread_resample_run:

            # Iterate over instrument symbol names
            for ins in self.instrument_exchange:
                
                # If the data feed hasn't begun, that is the length of raw_data is 0 
                if len(self.raw_data[ins]) == 0:

                    # Then continue
                    continue
                  
                # Create a temporary dataframe which is resampled from raw_data at the mentioned timeframe
                temp = pd.DataFrame({'open' : self.raw_data[ins]['ltp'].resample(self.timeframe).first(),
                                     'high' : self.raw_data[ins]['ltp'].resample(self.timeframe).max(),
                                     'low' : self.raw_data[ins]['ltp'].resample(self.timeframe).min(),
                                     'close' : self.raw_data[ins]['ltp'].resample(self.timeframe).last(),
                                     'volume' : self.raw_data[ins]['volume'].resample(self.timeframe).sum()
                                    })

                # Add a new column to the temporary dataframe to save the instrument ticker name
                temp['ticker'] = ins
                
                # Add a new column to the temporary dataframe to save the resampled date
                temp['date'] = temp.index.date
                
                # Add a new column to the temporary dataframe to save the resampled time
                temp['time'] = temp.index.time

                # Add the first row of the temp dataframe at that particular index in class attribute data
                self.data[ins].at[temp.index[0]] = temp.iloc[0]
                

                if (((datetime.datetime.now() - self.raw_data[ins].index[-1]) > self.obtain_timedelta('30S')) 
                    & ((datetime.datetime.now() - self.last_checked) > self.obtain_timedelta('2Min'))):
                    
                    self.data_strike = self.data_strike + 1
                    self.last_checked = datetime.datetime.now()
                    if self.data_strike > 5:
                        
                        telegram_bot.send('ALICE BLUE DATA DISCONNECTED')
                        print('ALICE BLUE DATA DISCONNECTED')
                        # exit()
                
                
                # If the datetime index of the last row of temporary dataframe is greater than 
                # the last row of class attribute data
                if temp.index[-1] > self.data[ins].index[-1]:

                    # Use the static method obtain_timedelta to find out what the next datetime index
                    # in class attribute data should be
                    next_tick = self.data[ins].index[-1] + self.obtain_timedelta(self.timeframe)

                    # Discard all data before the next_tick datetime index from raw_data in order to resample it again in 
                    # the next loop and extract latest resampled data from the first row of the temporary data formed 
                    # by resampling raw_data
                    self.raw_data[ins] = self.raw_data[ins][self.raw_data[ins].index > next_tick]
    
    # -----------------------------------------------------------
    # Start threading to run resampling in the background
    # -----------------------------------------------------------
    
    def start_thread(self):

        # Create new thread for resample_data method
        self.thread_resample = threading.Thread(target = self.resample_data)
        
        # Start running the thread
        self.thread_resample.start()
        
    # -----------------------------------------------------------
    # Start Live Data Streaming by subscribing to specified 
    # instruments and start the resampling thread in the background
    # Run this only once
    # -----------------------------------------------------------
    
    def get_live_data_initialise(self):
        
        # Subscribe to instruments and get their historical data
        # We have to make sure to run this only once and not anymore
        self.start_live_feed()
        self.start_thread()
       
    # -----------------------------------------------------------
    # Terminate the resampling thread in the background
    # -----------------------------------------------------------
    
    def terminate_thread(self):
            
        self.thread_resample_run = False
        self.thread_resample.join()


#### CLASS OMS

In [ ]:
class AmadeusOMS():
    
    def __init__(self):
        
        fyers_credentials = pd.read_csv('fyers_credentials.csv')

        self.app_id = fyers_credentials['appId'].values[0]
        self.app_secret = fyers_credentials['appSecret'].values[0]
        self.fyers_id = fyers_credentials['fyersId'].values[0]
        self.password = fyers_credentials['password'].values[0]
        self.pan_dob = fyers_credentials['pan'].values[0]
        self.last_checked = datetime.datetime.now()
        
        app_session = accessToken.SessionModel(self.app_id, self.app_secret)

        response = app_session.auth()

        authorization_code = response['data']['authorization_code']

        app_session.set_token(authorization_code)

        authorization_url = app_session.generate_token()
        
        login_url = "https://api.fyers.in/api/v1/token"

        login_headers = {
            "referer":authorization_url,
            "content-type":"application/json; charset=UTF-8",
            "origin":"https://api.fyers.in"
        }

        login_data = {
            "appId": self.app_id,
            "create_cookie":False,
            "fyers_id": self.fyers_id,
            "pan_dob": self.pan_dob,
            "password": self.password
        }

        login_request = requests.post(login_url, data=json.dumps(login_data), headers=login_headers)   
        
        self.token = json.loads(login_request.text)['Url'].split('access_token=')[1].split('&user_id')[0]

        is_async = False

        self.fyers = fyersModel.FyersModel(is_async)
        
        self.oms_strike = 0

        self.run_checker()
        
    def run_checker(self):
            
        self.run_check_thread = threading.Thread(target = self.run_checker_for_thread)
        self.run_check_thread.start()
        
    def stop_checker(self):
            
        self.run_checker_flag = False

    def run_checker_for_thread(self):

        self.run_checker_flag = True

        while ((self.run_checker_flag) &
               (datetime.datetime.now().time() > datetime.time(hour=9, minute=30)) &
               (datetime.datetime.now().time() < datetime.time(hour=15, minute=30)) &
               ((datetime.datetime.now() - self.last_checked) > self.obtain_timedelta('2Min'))):

            try:

                self.get_details()

            except:

                self.last_checked = datetime.datetime.now()
                self.oms_strike = self.oms_strike + 1

                if self.oms_strike > 5:

                    telegram_bot.send('OMS DISCONNECTED')
                    
    @staticmethod
    def obtain_timedelta(timeframe):

        # If the timeframe is in seconds, as indicated by the last element of the string being S
        if timeframe[-1] == 'S':
            
            # Set timedelta to seconds equal to the number specified before the last element
            td = datetime.timedelta(seconds = int(timeframe[:-1]))

        # Else If the timeframe is in minutes, as indicated by the last element of the string being T
        elif timeframe[-1] == 'T':
            
            # Set timedelta to minutes equal to the number specified before the last element
            td = datetime.timedelta(minutes = int(timeframe[:-1]))

        # Else If the timeframe is in minutes, as indicated by the last 3 elements of the string being min
        elif timeframe[-3:].lower() == 'min':
            
            # Set timedelta to minutes equal to the number specified before the last 3 elements
            td = datetime.timedelta(minutes = int(timeframe[:-3]))

        # Else If the timeframe is in hours, as indicated by the last element of the string being H
        elif timeframe[-1] == 'H':
            
            # Set timedelta to hours equal to the number specified before the last element
            td = datetime.timedelta(hours = int(timeframe[:-1]))

        # Else If the timeframe is in days, as indicated by the last element of the string being D
        elif timeframe[-1] == 'D':
            
            # Set timedelta to days equal to the number specified before the last element
            td = datetime.timedelta(days = int(timeframe[:-1]))

        # Else If the timeframe is in weeks, as indicated by the last element of the string being W
        elif timeframe[-1] == 'W':
            
            # Set timedelta to weeks equal to the number specified before the last element
            td = datetime.timedelta(weeks = int(timeframe[:-1]))

        # If none of the above conditions are met
        else:
            
            # Set timedelta to None
            td = None

        # Return the calculated timedelta
        return td
        
    def get_details(self, details = 'account'):
        
        '''
        Provides Required Details
        Input : Specify what details are required 
                (account / holdings / funds / order book / positions / trade book)
        
        '''
        
        if details == 'account':
            
            self.acc_details = self.fyers.get_profile(token = self.token)['data']['result']

            return self.acc_details
        
        elif details == 'holdings':
            
            return self.fyers.holdings(token = self.token)

        elif details == 'funds':
            
            return self.fyers.funds(token = self.token).get('data').get('fund_limit')[-1]

        elif details == 'order book':
            
            return self.fyers.orders(token = self.token).get('data')

        elif details == 'positions':
            
            return self.fyers.positions(token = self.token).get('data')
        
        elif details == 'trade book':
            
            return self.fyers.tradebook(token = self.token).get('data')
        
        
    def get_all_symbols(self, exchange = 'NSE_CM'):
        
        '''
        Get all symbols from the required exchange
        Input : Specify the exchange name
                ('NSE_CM'/'NSE_FO'/'NSE_CD'/'MCX_COM')
        
        '''
        
        self.all_symbols_list = pd.read_csv(f"http://public.fyers.in/sym_details/{exchange}.csv",
                                            names = [
                                                'Fyers Token', 'Symbol Details', 'Exchange Instrument',
                                                'Minimum lot size', 'Tick size', 'ISIN', 'Trading Session',
                                                'Last update date', 'Expiry date', 'Symbol ticker',
                                                'Exchange', 'Segment', 'Scrip code']
                                            )
        
        return self.all_symbols_list

    def place_market_order(self, symbol, qty=1, side=1, productType = "INTRADAY", validity = "IOC",
                          offlineOrder = "False", stopLoss = 0, takeProfit = 0):

        '''
        Places Market Order

        symbol : Provide Symbol Name

        qty : Provide Quantity
            default : 1

        side : Provide 1 for Buy and -1 for Sell
            default : 1

        productType : 
            CNC - For equity only
            INTRADAY - Applicable for all segments.
            MARGIN - Applicable only for derivatives
            CO - Cover Order
            BO - Bracket Order
                default : INTRADAY

        validity:
            IOC - Immediate or Cancel
            DAY - Valid till the end of the day
                default : IOC

        offlineOrder: 
            False => When market is open
            True => When placing AMO order
                default : False

        stopLoss : Provide valid price for CO and BO orders

        takeProfit: Provide valid price for BO orders
        '''

        response = self.fyers.place_orders(token=self.token,
                                data={
                                    "symbol": symbol,
                                    "qty": qty,
                                    "type": 2,
                                    "side": side,
                                    "productType": productType,
                                    "limitPrice": 0,
                                    "stopPrice": 0,
                                    "disclosedQty": 0,
                                    "validity": validity,
                                    "offlineOrder": offlineOrder,
                                    "stopLoss": stopLoss,
                                    "takeProfit": takeProfit
                                })
        
        response_code = {}

        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('message')})

        elif response.get('code') == 200:

            response_code.update({'code': 200, 'id': response.get('data').get('id')})

        return response_code
    

    def place_limit_order(self, symbol, limitPrice, qty=1, side=1, productType = "INTRADAY", validity = "IOC",
                          offlineOrder = "False", stopLoss = 0, takeProfit = 0):

        '''
        Places Market Order

        symbol : Provide Symbol Name

        qty : Provide Quantity
            default : 1

        side : Provide 1 for Buy and -1 for Sell
            default : 1

        limitPrice- : Provide valid price for Limit and Stoplimit orders

        productType : 
            CNC - For equity only
            INTRADAY - Applicable for all segments.
            MARGIN - Applicable only for derivatives
            CO - Cover Order
            BO - Bracket Order
                default : INTRADAY

        validity:
            IOC - Immediate or Cancel
            DAY - Valid till the end of the day
                default : IOC

        offlineOrder: 
            False => When market is open
            True => When placing AMO order
                default : False

        stopLoss : Provide valid price for CO and BO orders

        takeProfit: Provide valid price for BO orders
        '''

        response = self.fyers.place_orders(token=self.token,
                                data={
                                    "symbol": symbol,
                                    "qty": qty,
                                    "type": 1,
                                    "side": side,
                                    "productType": productType,
                                    "limitPrice": limitPrice,
                                    "stopPrice": 0,
                                    "disclosedQty": 0,
                                    "validity": validity,
                                    "offlineOrder": offlineOrder,
                                    "stopLoss": stopLoss,
                                    "takeProfit": takeProfit
                                })
        
        response_code = {}

        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('message')})

        elif response.get('code') == 200:

            response_code.update({'code': 200, 'id': response.get('data').get('id')})

        return response_code

    def place_slm_order(self, symbol, stopPrice, qty=1, side=1, productType = "INTRADAY", validity = "IOC",
                          offlineOrder = "False", stopLoss = 0, takeProfit = 0):

        '''
        Places Market Order

        symbol : Provide Symbol Name

        qty : Provide Quantity
            default : 1

        side : Provide Buy or 1 / Sell or -1
            default : 1

        stopPrice : Provide valid price for Stop and Stoplimit orders

        productType : 
            CNC - For equity only
            INTRADAY - Applicable for all segments.
            MARGIN - Applicable only for derivatives
            CO - Cover Order
            BO - Bracket Order
                default : INTRADAY

        validity:
            IOC - Immediate or Cancel
            DAY - Valid till the end of the day
                default : IOC

        offlineOrder: 
            False => When market is open
            True => When placing AMO order
                default : False

        stopLoss : Provide valid price for CO and BO orders

        takeProfit: Provide valid price for BO orders
        '''

        response = self.fyers.place_orders(token=self.token,
                                data={
                                    "symbol": symbol,
                                    "qty": qty,
                                    "type": 3,
                                    "side": side,
                                    "productType": productType,
                                    "limitPrice": 0,
                                    "stopPrice": stopPrice,
                                    "disclosedQty": 0,
                                    "validity": validity,
                                    "offlineOrder": offlineOrder,
                                    "stopLoss": stopLoss,
                                    "takeProfit": takeProfit
                                })
        
        response_code = {}

        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('message')})

        elif response.get('code') == 200:

            response_code.update({'code': 200, 'id': response.get('data').get('id')})

        return response_code
        

    def place_sll_order(self, symbol, limit, stopPrice, qty=1, side=1, productType = "INTRADAY", validity = "IOC",
                          offlineOrder = "False", stopLoss = 0, takeProfit = 0):

        '''
        Places Market Order

        symbol : Provide Symbol Name

        qty : Provide Quantity
            default : 1

        side : Provide Buy or 1 / Sell or -1
            default : 1

        limitPrice : Provide valid price for Limit and Stoplimit orders

        stopPrice : Provide valid price for Stop and Stoplimit orders

        productType : 
            CNC - For equity only
            INTRADAY - Applicable for all segments.
            MARGIN - Applicable only for derivatives
            CO - Cover Order
            BO - Bracket Order
                default : INTRADAY

        validity:
            IOC - Immediate or Cancel
            DAY - Valid till the end of the day
                default : IOC

        offlineOrder: 
            False => When market is open
            True => When placing AMO order
                default : False

        stopLoss : Provide valid price for CO and BO orders

        takeProfit: Provide valid price for BO orders
        '''

        response = self.fyers.place_orders(token=self.token,
                                data={
                                    "symbol": symbol,
                                    "qty": qty,
                                    "type": 4,
                                    "side": side,
                                    "productType": productType,
                                    "limitPrice": limitPrice,
                                    "stopPrice": stopPrice,
                                    "disclosedQty": 0,
                                    "validity": validity,
                                    "offlineOrder": offlineOrder,
                                    "stopLoss": stopLoss,
                                    "takeProfit": takeProfit
                                })
        
        response_code = {}

        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('message')})

        elif response.get('code') == 200:

            response_code.update({'code': 200, 'id': response.get('data').get('id')})

        return response_code
    
    
    def get_order_status(self, order_id):
        
        '''
        Get the status of a particular order
        
        Input : order_id : Provide Order Id in string format
        '''
        
        response = self.fyers.order_status(token=self.token, data={"id": "52008017215"})
        
        status_mapping = {
                            1: 'Cancelled',
                            2: 'Filled',
                            3: 'Not used currently',
                            4: 'Transit',
                            5: 'Rejected',
                            6: 'Pending'
                        }

        order_type_mapping = {
                                1: 'Market order',
                                2: 'Limit order',
                                3: 'Stop order (SL-M)',
                                4: 'Stop limit order(SL-L)'
                             }

        response_code = {}

        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('data').get('orderDetails').get('message')})

        elif response.get('code') == 200:

            response = response.get('data').get('orderDetails')
            response['status'] = status_mapping[response['status']]
            response['type'] = order_type_mapping[response['type']]
            attributes = [
                            'symbol', 'qty', 'orderDateTime', 'id', 'stopPrice', 'instrument',
                            'remainingQuantity', 'filledQty', 'limitPrice', 'offlineOrder',
                            'productType', 'type', 'tradedPrice'
                         ]
            response_code = {k:v for k, v in response.items() if k in attributes}

        return response_code
    
    def exit_positions(self, symbol=''):
        
        '''
        Places Market Order

        Input : symbol - Provide Symbol Name
        '''
    
        response = self.fyers.exit_positions(token = self.token, data = {"id" : symbol})
        
        response_code = {}
        
        if response.get('code') == 400:

            response_code.update({'code': 400, 'message': response.get('message')})

        elif response.get('code') == 200:

            response_code.update({'code': 200, 'id': response.get('data').get('id')})

        return response_code

#### ORB

In [ ]:
class ORB:
    
    def __init__(self, instrument, exchange, fyers_symbol_ticker, quantity, data, oms):
        
        self.instrument = instrument
        self.exchange = exchange
        self.fyers_symbol_ticker = fyers_symbol_ticker
        self.data = data
        self.oms = oms
        self.quantity = quantity
        self.long_trade = False
        self.short_trade = False
        self.position = 0
        self.today = pd.Timestamp(datetime.date.today())
        self.prices_calculated_flag = False
        
        if self.exchange == 'MCX':
            self.market_close = datetime.time(hour=23, minute=30)
        else:
            self.market_close = datetime.time(hour=15, minute=30)
            
            
    def create_price_levels(self):
        
        self.index = self.data.index[-1]
        self.row = self.data.iloc[-1]

        if ((self.index.date() == self.today) & 
           (self.index.time() > datetime.time(hour=9, minute=30)) & 
           (self.prices_calculated_flag == False)):

            self.candle = self.data[(self.data['time'] >= datetime.time(hour=9, minute=15)) & 
                                                              (self.data['time'] <= datetime.time(hour=9, minute=30)) &
                                                              (self.data['date'] == self.today)]

            self.long_bo_level = self.candle['high'].max()
            self.short_bo_level = self.candle['low'].min()
            
            self.prices_calculated_flag = True
            
            
    def run_strategy(self):

        self.thread_run_strategy = threading.Thread(target = self.run_strategy_for_thread)
        self.thread_run_strategy.start()
        
    def terminate_signals(self):
            
        self.run_strategy_for_thread = False
        
    def run_strategy_for_thread(self):
        
        self.run_strategy_thread_flag = True
        
        while self.run_strategy_thread_flag:
            
            self.index = self.data.index[-1]
            self.ltp = self.data.iloc[-1]['close']
            
            if ((self.index.date() == self.today) &
                (self.index.time() > datetime.time(hour=9, minute=30))):
                
                if self.prices_calculated_flag == False:
                    self.create_price_levels()
                    
                # If LTP >= long_bo_level AND not long_trade AND position == 0:
                    # Open long trade
                    # Fire 1*Qty buy market order
                    # position = 1

                # If LTP <= short_bo_level AND not short_trade AND position == 0:
                    # Open short trade
                    # Fire 1*Qty sell market order
                    # position = -1

                # If LTP >= long_bo_level AND not long_trade AND position == -1:
                    # SAR the short trade
                    # Fire 2*Qty buy market order
                    # position = 1

                # If LTP <= short_bo_level AND not short_trade AND position == 1:
                    # SAR the long trade
                    # Fire 2*Qtysell market order
                    # position = -1

                # If LTP <= short_bo_level AND short_trade AND position == 1:
                    # Close long position
                    # Fire 1*Qty sell market order
                    # position = 0

                # If LTP >= long_bo_level AND long_trade AND position == -1:
                    # Close short position
                    # Fire 1*Qty buy market order
                    # position = 0
                    
                ### AT 15.15 ###

                    # If position != 0:
                        # Fire order to close position
                        # -position*Qty

                if (self.ltp >= self.long_bo_level) & (not self.long_trade) & (self.position == 0):
                    
                    self.order = self.oms.place_market_order(symbol = self.fyers_symbol_ticker, 
                                                        qty=self.quantity, side=1, productType = "INTRADAY", 
                                                        validity = "IOC", offlineOrder = "False", stopLoss = 0, 
                                                        takeProfit = 0)
                    
                    self.long_trade = True
                    self.position = 1
                    telegram_bot.send(f'PLACING LONG TRADE AT {self.ltp}')
                    
                if (self.ltp <= self.short_bo_level) & (not self.short_trade) & (self.position == 0):
                    
                    self.order = self.oms.place_market_order(symbol = self.fyers_symbol_ticker, 
                                                        qty=self.quantity, side=-1, productType = "INTRADAY", 
                                                        validity = "IOC", offlineOrder = "False", stopLoss = 0, 
                                                        takeProfit = 0)
                    
                    self.short_trade = True
                    self.position = -1
                    telegram_bot.send(f'PLACING SHORT TRADE AT {self.ltp}')
                    
                if (self.ltp >= self.long_bo_level) & (not self.long_trade) & (self.position == -1):
                    
                    self.order = self.oms.exit_positions(symbol = self.fyers_symbol_ticker)
                    
                    self.order = self.oms.place_market_order(symbol = self.fyers_symbol_ticker, 
                                                        qty=self.quantity, side=1, productType = "INTRADAY", 
                                                        validity = "IOC", offlineOrder = "False", stopLoss = 0, 
                                                        takeProfit = 0)
                    
                    self.long_trade = True
                    self.short_trade = False
                    self.position = 1
                    telegram_bot.send(f'PLACING LONG TRADE AT {self.ltp}')
                    
                if (self.ltp <= self.short_bo_level) & (not self.short_trade) & (self.position == 1):    
                    
                    self.order = self.oms.exit_positions(symbol = self.fyers_symbol_ticker)
                    
                    self.order = self.oms.place_market_order(symbol = self.fyers_symbol_ticker, 
                                                        qty=self.quantity, side=-1, productType = "INTRADAY", 
                                                        validity = "IOC", offlineOrder = "False", stopLoss = 0, 
                                                        takeProfit = 0)
                    
                    self.short_trade = True
                    self.long_trade = False
                    self.position = -1
                    telegram_bot.send(f'PLACING SHORT TRADE AT {self.ltp}')
                    
                    
                if ((self.position != 0) & (self.index.time() >= datetime.time(hour=self.market_close.hour, minute=self.market_close.minute-15))):
                    
                    self.order = self.oms.exit_positions(symbol = self.fyers_symbol_ticker)
                    self.run_strategy_thread_flag = False
                    telegram_bot.send(f'EXITING ALL POSITIONS AT {self.ltp}')
                    

***

#### MAIN FUNCTION - ORB

In [ ]:
## TO GET FYERS SYMBOL TICKER ###
# amadeus_oms = AmadeusOMS()

# df = amadeus_oms.get_all_symbols()

# df[df['Symbol Details'].apply(lambda x : True if 'GOLDPETAL' in x else False)]

In [ ]:
'''
input_instruments = {
    'RELIANCE': {
        'exchange': 'NSE',
        'quantity': 1,
        'fyers_symbol_ticker': 'NSE:RELIANCE-EQ',
    }
}
'''

In [ ]:
input_instruments = {
    'GOLDPETAL AUG FUT': {
        'exchange': 'MCX',
        'quantity': 1,
        'fyers_symbol_ticker': 'MCX:GOLDPETAL20AUGFUT',
    }
}

In [ ]:
try:
    amadeus_data = AmadeusData('1min', input_instruments)

except Exception as e:
    telegram_bot.send(f'Error has occured while creating Amadeus Data Client object \n{e}')
    print(f'Error has occured while creating Amadeus Data Client objectd \n{e}')
    exit()

In [ ]:
try:
    amadeus_oms = AmadeusOMS()

except Exception as e:      
    telegram_bot.send(f'Error has occured while creating Amadeus OMS Client object \n{e}')
    print(f'Error has occured while creating Amadeus OMS Client object \n{e}')
    exit()

In [ ]:
# sleeper(9,25)

In [ ]:
try:
    amadeus_data.get_live_data_initialise()

except Exception as e:
    telegram_bot.send(f'Error has occured while trying to get Live Data Feed \n{e}')
    print(f'Error has occured while trying to get Live Data Feed \n{e}')
    exit()

In [ ]:
print(amadeus_data.last_checked)

In [ ]:
amadeus_data.socket_opened

In [ ]:
amadeus_data.data['GOLDPETAL AUG FUT']

In [ ]:
amadeus_data.raw_data['GOLDPETAL AUG FUT']

In [ ]:
orb_objects_dict = {}

for instrument in input_instruments:
    
    orb_objects_dict[instrument] = ORB(instrument = instrument, 
                          exchange = input_instruments[instrument].get('exchange'), 
                          fyers_symbol_ticker = input_instruments[instrument].get('fyers_symbol_ticker'), 
                          quantity = input_instruments[instrument].get('quantity'),
                          data = amadeus_data.data[instrument],
                          oms = amadeus_oms
                          )
    
    orb_objects_dict[instrument].run_strategy()

In [ ]:
orb_objects_dict['RELIANCE'].position

In [ ]:
orb_objects_dict['RELIANCE'].short_bo_level

In [ ]:
orb_objects_dict['RELIANCE'].long_bo_level

In [ ]:
orb_objects_dict['RELIANCE'].long_trade

In [ ]:
orb_objects_dict['RELIANCE'].short_trade

In [ ]:
orb_objects_dict['RELIANCE'].order

***